### <p style="background-color:white;font-family:timesnewroman;color:lightseagreen;font-size:200%;border-radius:20px 60px;">Introduction</p>

 #### In Retail Industry and chain of stores one of the biggest issue they face are supply chain management. The component of supply chain management (SCM) involved with determining how best to fulfill the requirements created from the Demand Plan.

 *It's objective is to balance supply and demand in a manner that achieves the financial and service objectives of the enterprise.*

  If we look into the case of a retail chain stores one of the basic case is to know the demand of products that are sold in the store. If the decision making authority know whats the demand of each products for a week or month, they would be able to plan the supply chain accordingly. If that is possible this would save a lot of money for them because they don't have to overstock or can plan their Logistics accordingly.


### <p style="background-color:white;font-family:calibri;color:lightseagreen;font-size:200%;border-radius:20px 60px;">Data</p>

### There are 3 Datasets :

#### Stores:
- Store: The store number. Range from 1–45.
- Type: Three types of stores ‘A’, ‘B’ or ‘C’.
- Size: Sets the size of a Store would be calculated by the no. of products available in the particular store ranging from 34,000 to 210,000.

***primary key is Store***

#### Sales:
    -Date: The date of the week where this observation was taken.
    -Weekly_Sales: The sales recorded during that Week.
    -Store: The store which observation in recorded 1–45
    -Dept: One of 1–99 that shows the department.
    -IsHoliday: Boolean value representing a holiday week or not.

***primary key is a combination of (Store,Dept,Date).***

#### Features:

    -Temperature: Temperature of the region during that week.
    -Fuel_Price: Fuel Price in that region during that week.
    -MarkDown1:5 : Represents the Type of markdown and what quantity was available during that week.
    -CPI: Consumer Price Index during that week.
    -Unemployment: The unemployment rate during that week in the region of the store.

***primary key here is a combination of (Store,Date)***

![](https://i.imgur.com/XuDXqGU.png)


### <p style="background-color:white;font-family:calibri;color:lightseagreen;font-size:200%;border-radius:20px 60px;">Plan of Action</p>

    1. We will perform dertailed EDA & employ several algortihms we learned in class
    2. Next we will build the following models to predict future sales.


 **List of Models -**

     1. XXXXXXXXXX -- Please fill in
     2. YYYYYYYYYY -- Please fill in
     3. ZZZZZZZZZZ -- Please fill in
     4. AAAAAAAAAA -- Please fill in
     5. Gradient Booster
     6. Extreme Gradient Boooster
     7. Fully Connected NN


In [ ]:
# importing basic packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import datetime # manipulating date formats
# Viz
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for plots
import mlxtend
%matplotlib inline

# settings
import warnings
warnings.filterwarnings("ignore")

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:150%;text-align:center;border-radius:20px 60px;">Dataset Importing and Querying</p>

    We will load all datasets and merge them into one dataset that gives whole data.

In [ ]:
# Reading the data using pandas dataframe
features = pd.read_csv('/Users/cwndroha/Downloads/walmart-recruiting-store-sales-forecasting/features/features.csv')
train    = pd.read_csv('/Users/cwndroha/Downloads/walmart-recruiting-store-sales-forecasting/train/train.csv')
stores   = pd.read_csv('/Users/cwndroha/Downloads/walmart-recruiting-store-sales-forecasting/stores.csv')

- Printing sample data

In [ ]:
df_names=['features','stores','train']
df_list=[features,stores,train]
for i in range(3):
    print('--'*25)
    print(f'Dataframe {df_names[i]} has {df_list[i].shape[0]} rows and {df_list[i].shape[1]} columns.')
    print('--'*25)
    display(df_list[i].head(5))

- Merging Data

In [ ]:
df = train.merge(stores, how='left').merge(features, how='left')

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Data Cleaning</p>

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Week'] = pd.to_datetime(df['Date']).dt.week
df['Day'] = pd.to_datetime(df['Date']).dt.day
df.replace({'A': 1, 'B': 2,'C':3},inplace=True)
df['IsHoliday'] = df['IsHoliday'].apply(lambda x: 1 if x == True else 0)
df.head(50)

In [ ]:
print(df.shape)

In [ ]:
# Want to check the date column is in object format or datetime
df.dtypes

- Calculating missing values

In [ ]:
print('Percentages of missing values in merged dataframe.')
(100*df.isna().sum()/df.shape[0]).sort_values()

In [ ]:
# Imputing the null values in markdown columns with 0
df.iloc[:,9:14]=df.iloc[:,9:14].fillna(0)

In [ ]:
print('Percentages of missing values in merged dataframe.')
(100*df.isna().sum()/df.shape[0]).sort_values()

- Creating Markdown variable which is equal to 1 if any markdown is equal to 1, 0 otherwise

In [ ]:
df['markdown'] = df.iloc[:,9:14].sum(axis=1)
df['markdown'] = df['markdown'].apply(lambda x:0 if x==0 else 1)

- Moving Target Variable to End of DF

In [ ]:
cols = list(df.columns) # List of colmuns of dataframe df
cols.pop(cols.index('Weekly_Sales')) # Moving Target Variable to end of DF to better read Corr Matrix
df = df[cols+['Weekly_Sales']] # Moving Target Variable to end of DF to better read Corr Matrix
df.head(5)

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), cmap='summer', annot=True)
plt.title('Correlation Matrix -- feature importance', fontsize=30)
plt.show()

In [ ]:
from scipy.stats import spearmanr # For Spearman correlation

cols=df.columns
cols=cols.drop(['Date'])

for col in cols[:-1]:
    rho, p = spearmanr(df[col].values, df['Weekly_Sales'].values)
    print('Spearman correlation between Weekly_Sales and %s is %s' %(col, round(rho,4)))

In [ ]:
cr = df.corr()
plt.figure(figsize=(10,10), dpi=80)
plt.title('Multicollinearity', fontsize=30)
sns.heatmap(cr[(cr>=0.6)|(cr<=-0.6)],annot=True,cmap='coolwarm')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
df.groupby('Date')['Weekly_Sales'].mean().plot()
plt.grid()
plt.title('Average weekly Sales of the company across all stores in given timeframe', fontsize=18)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Date', fontsize=16);

In [ ]:
weekly_sales = df.groupby(['Year','Week']).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2010 = df.loc[df['Year']==2010].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2011 = df.loc[df['Year']==2011].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2012 = df.loc[df['Year']==2012].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})
plt.figure(figsize=(15, 5))
sns.lineplot(weekly_sales2010['Weekly_Sales']['mean'].index, weekly_sales2010['Weekly_Sales']['mean'].values)
sns.lineplot(weekly_sales2011['Weekly_Sales']['mean'].index, weekly_sales2011['Weekly_Sales']['mean'].values)
sns.lineplot(weekly_sales2012['Weekly_Sales']['mean'].index, weekly_sales2012['Weekly_Sales']['mean'].values)

plt.grid()
plt.title('Average weekly Sales by Year.', fontsize=18)
plt.xticks(np.arange(1, 53, step=1))
plt.legend(['2010', '2011', '2012'])
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
df[df['Type']==1].groupby('Week').mean()['Weekly_Sales'].plot()
df[df['Type']==2].groupby('Week').mean()['Weekly_Sales'].plot()
df[df['Type']==3].groupby('Week').mean()['Weekly_Sales'].plot()
plt.grid()
plt.title('Average weekly Sales of the company by type of the store.', fontsize=18)
plt.legend(['Type A', 'Type B', 'Type C'], loc='best', fontsize=16)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Weeks', fontsize=16);

In [ ]:
def av_sales_plotter(str):
    plt.figure(figsize=(20,5))
    df.groupby(str).mean()['Weekly_Sales'].sort_values().plot(kind='bar',color='crimson')
    plt.title(f'Sales of each {str}.', fontsize=18)
    plt.ylabel('Sales', fontsize=16)
    plt.xlabel(str, fontsize=16)
    plt.tick_params(axis='x', labelsize=14)

In [ ]:
av_sales_plotter('Dept')

In [ ]:
av_sales_plotter('Store')

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x='Store',y='Size',data=stores,order=stores.sort_values('Size')['Store'].tolist())
plt.title('Sizes of all the stores.',fontsize=15)
plt.tight_layout();

In [ ]:
def scatter(dataset, column):
    plt.figure(figsize=(15,5))
    plt.scatter(df[column] , df['Weekly_Sales'], color = 'crimson')
    plt.title('Scatter Plot of Actual Sales.', fontsize=15)
    plt.ylabel('Weekly Sales')
    plt.xlabel(column)

In [ ]:
scatter(df, 'Date')

In [ ]:
# importing relevant libraries

from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

# Importing basic tensorflow modules

from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
from tensorflow.keras.utils import plot_model

import torch

# Importing 'xgboost'
import xgboost as xgb

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Gradient Boosting Regressor - I.</p>

In [ ]:
df1=df.filter(['Store','Dept','IsHoliday','Markdown','Size','Week','Month','Weekly_Sales'])

# Defining the features and the target of the model

features = df1.columns[:-1]
target = df1.columns[-1]

X = df1[features].values
y = df1[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
# Importing 'GradientBoostingRegressor'

from sklearn.ensemble import GradientBoostingRegressor

gb_reg = GradientBoostingRegressor() # Instantiating GradientBoostingRegressor
gb_reg.fit(X_train, y_train) # Fitting the train data

In [ ]:
# Finding the predictions of gradient boosting regressor for train and test subsets
train_y_pred = gb_reg.predict(X_train)
test_y_pred = gb_reg.predict(X_test)

In [ ]:
r2_train_score = gb_reg.score(X_train, y_train) # Calculating R^2 score for train
r2_test_score = gb_reg.score(X_test, y_test) # Calculating R^2 score for test
print('R^2 score for train dataset = ', round(r2_train_score, 4), '\n')
print('R^2 score for test dataset = ', round(r2_test_score, 4), '\n')

In [ ]:
# Calculating the feature importance

feature_importance = gb_reg.feature_importances_
feature_importance = 100.0*(feature_importance/np.sum(feature_importance))
# Displaying the relative feature importance by a horizontal bar chart
sorted_idx = np.argsort(feature_importance)
pos=np.arange(sorted_idx.shape[0])+0.5
plt.figure(figsize=(7, 7))
plt.title("GB_REG -- Feature Importance w/o Hyperparameters")
plt.xlabel('Relative Importance')
plt.barh(pos, feature_importance[sorted_idx], color='crimson', align="center")
plt.yticks(pos, df1.columns[:-1][sorted_idx])
plt.show()

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Gradient Boosting Regressor - II. Intro Hyperparameters</p>

In [ ]:
# Setting new hyperparameters for gradient boosting regressor
gb_params = {'n_estimators': 1000, 'max_depth': 6, 'min_samples_split': 15, 'learning_rate': 0.01}

gb_reg = GradientBoostingRegressor(**gb_params) # Applying the new hyperparameters
gb_reg.fit(X_train, y_train) # Fitting the train data

In [ ]:
# Finding the predictions of gradient boosting regressor for train and test subsets
train_y_pred = gb_reg.predict(X_train)
test_y_pred = gb_reg.predict(X_test)

In [ ]:
r2_train_score = gb_reg.score(X_train, y_train) # Calculating R^2 score for train
r2_test_score = gb_reg.score(X_test, y_test) # Calculating R^2 score for test
print('R^2 score for train dataset = ', round(r2_train_score, 4), '\n')
print('R^2 score for test dataset = ', round(r2_test_score, 4), '\n')

In [ ]:
# Calculating the feature importance

feature_importance = gb_reg.feature_importances_
feature_importance = 100.0*(feature_importance/np.sum(feature_importance))
# Displaying the relative feature importance by a horizontal bar chart
sorted_idx = np.argsort(feature_importance)
pos=np.arange(sorted_idx.shape[0])+0.5
plt.figure(figsize=(7, 7))
plt.title("GB_REG -- Feature Importance w/ Hyperparameters")
plt.xlabel('Relative Importance')
plt.barh(pos, feature_importance[sorted_idx], color='crimson', align="center")
plt.yticks(pos, df1.columns[:-1][sorted_idx])
plt.show()

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Gradient Boosting Regressor - III. Tuned Hyperparameters, Low end</p>

In [ ]:
# Importing 'GridSearchCV' from 'sklearn'

#from sklearn.model_selection import GridSearchCV

# Specifying the hyperparameters

#param_grid = {'n_estimators': [500,1000],
#               'max_depth': [5,10],
#               'min_samples_split': [10,15]}

#grid = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=-1, verbose=3)

#grid.fit(X_train,y_train)

#print('\n Best choices for hyperparameters:\n', grid.best_params_) # Presenting best choice

#grid_predictions = grid.predict(X_test) # Compute the predictions of SVM with best choice of hyperparameters

In [ ]:
# Setting new hyperparameters for gradient boosting regressor
gb_params = {'n_estimators': 500, 'max_depth': 10, 'min_samples_split': 15, 'learning_rate': 0.01}

gb_reg = GradientBoostingRegressor(**gb_params) # Applying the new hyperparameters
gb_reg.fit(X_train, y_train) # Fitting the train data

In [ ]:
# Finding the predictions of gradient boosting regressor for train and test subsets
train_y_pred = gb_reg.predict(X_train)
test_y_pred = gb_reg.predict(X_test)

In [ ]:
r2_train_score = gb_reg.score(X_train, y_train) # Calculating R^2 score for train
r2_test_score = gb_reg.score(X_test, y_test) # Calculating R^2 score for test
print('R^2 score for train dataset = ', round(r2_train_score, 4), '\n')
print('R^2 score for test dataset = ', round(r2_test_score, 4), '\n')

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Gradient Boosting Regressor - IV. Tuned Hyperparameters, High End</p>

In [ ]:
# Importing 'GridSearchCV' from 'sklearn'

#from sklearn.model_selection import GridSearchCV

# Specifying the hyperparameters

#param_grid = {'n_estimators': [1000,2000],
#               'max_depth': [10,15],
#               'min_samples_split': [15,20]}

#grid = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=-1, verbose=3)

#grid.fit(X_train,y_train)

#print('\n Best choices for hyperparameters:\n', grid.best_params_) # Presenting best choice

#grid_predictions = grid.predict(X_test) # Compute the predictions of SVM with best choice of hyperparameters

In [ ]:
# Setting new hyperparameters for gradient boosting regressor
gb_params = {'n_estimators': 1000, 'max_depth': 10, 'min_samples_split': 20, 'learning_rate': 0.01}

gb_reg = GradientBoostingRegressor(**gb_params) # Applying the new hyperparameters
gb_reg.fit(X_train, y_train)

In [ ]:
# Finding the predictions of gradient boosting regressor for train and test subsets
train_y_pred = gb_reg.predict(X_train)
test_y_pred = gb_reg.predict(X_test)

In [ ]:
r2_train_score = gb_reg.score(X_train, y_train) # Calculating R^2 score for train
r2_test_score = gb_reg.score(X_test, y_test) # Calculating R^2 score for test

print('R^2 score for train dataset = ', round(r2_train_score, 4), '\n')
print('R^2 score for test dataset = ', round(r2_test_score, 4), '\n')

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Extreme Gradient Boosting Regressor - I. Feature Selection</p>

In [ ]:
df2=df.filter(['Store','Dept','Size','Week','Weekly_Sales'])

# Defining the features and the target of the model

features = df2.columns[:-1]
target = df2.columns[-1]

X = df2[features].values
y = df2[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
# Instantiate 'XGBRegressor'
xgb_reg = xgb.XGBRegressor(n_estimators=700, max_depth=6, eta=0.04, subsample=0.8)

xgb_reg.fit(X_train, y_train) # Fitting the train data

In [ ]:
# Finding the predictions of XGBoost regressor for train and test subsets
train_y_pred = xgb_reg.predict(X_train)
test_y_pred = xgb_reg.predict(X_test)

In [ ]:
r2_train_score = xgb_reg.score(X_train, y_train) # Calculating R^2 score for train
r2_test_score = xgb_reg.score(X_test, y_test) # Calculating R^2 score for test
print('R^2 score for train dataset = ', round(r2_train_score, 4), '\n')
print('R^2 score for test dataset = ', round(r2_test_score, 4), '\n')

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Extreme Gradient Boosting Regressor - II. Hyperparameter Tuning</p>

In [ ]:
# Specifying the hyperparameters

#param_grid = {'n_estimators': [500,1000],
               #'max_depth': [5,10,15]}

#grid = GridSearchCV(XGBRegressor(), param_grid, n_jobs=-1, verbose=3)

#grid.fit(X_train,y_train)

#print('\n Best choices for hyperparameters:\n', grid.best_params_) # Presenting best choice

#grid_predictions = grid.predict(X_test) # Compute the predictions of SVM with best choice of hyperparameters

In [ ]:
# Instantiate 'XGBRegressor'
xgb_reg = xgb.XGBRegressor(n_estimators=500, max_depth=10, eta=0.04, subsample=0.8)

xgb_reg.fit(X_train, y_train) # Fitting the train data

In [ ]:
# Finding the predictions of XGBoost regressor for train and test subsets
train_y_pred = xgb_reg.predict(X_train)
test_y_pred = xgb_reg.predict(X_test)

In [ ]:
r2_train_score = xgb_reg.score(X_train, y_train) # Calculating R^2 score for train
r2_test_score = xgb_reg.score(X_test, y_test) # Calculating R^2 score for test
print('R^2 score for train dataset = ', round(r2_train_score, 4), '\n')
print('R^2 score for test dataset = ', round(r2_test_score, 4), '\n')

In [ ]:
# Calculating the feature importance

feature_importance = xgb_reg.feature_importances_
feature_importance = 100.0*(feature_importance/np.sum(feature_importance))
# Displaying the relative feature importance by a horizontal bar chart
sorted_idx = np.argsort(feature_importance)
pos=np.arange(sorted_idx.shape[0])+0.5
plt.figure(figsize=(7, 7))
plt.title("XGB_REG -- Feature Importance")
plt.xlabel('Relative Importance')
plt.barh(pos, feature_importance[sorted_idx], color='crimson', align="center")
plt.yticks(pos, df1.columns[:-1][sorted_idx])
plt.show()

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Fully Connected NN - I. XXX</p>

In [ ]:
df2=df.filter(['Store','Dept','Weekly_Sales'])
df2_1=df2[df2['Store']==20] #limit to store
df2_2=df2_1[df2_1['Dept']==92] #limit to store and dept
# Defining the features and the target of the model

features = df2.columns[:-1]
target = df2.columns[-1]

X = df2[features].values
y = df2[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
df2=df.filter(['Store','Dept','Size','Week','Weekly_Sales'])

# Defining the features and the target of the model

features = df2.columns[:-1]
target = df2.columns[-1]

X = df2[features].values
y = df2[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
# Creating a simple function that builds a fully connected neural network

def create_model(n_neurons, learning_rate):
    model = Sequential()
    model.add(Dense(n_neurons, input_dim=X_train.shape[1], activation='tanh')) # Layer 1 + activation
    model.add(Dense(n_neurons, activation='tanh'))                             # Layer 2 + activation
    model.add(Dense(n_neurons, activation='tanh'))                             # Layer 3 + activation
    model.add(Dense(len(y_train.shape)))                                          # Output layer
    adam = Adam(learning_rate=learning_rate)                                      # Choice of optimizer
    # Compiling the model by specifying the choice of loss function ('mae'), and performance metrics
    model.compile(loss='mae', optimizer=adam, metrics=['mae', tfa.metrics.r_square.RSquare()])
    return model

In [ ]:
# Creating a model with 64 neurons and learning rate 0.001

#del model
model = create_model(n_neurons=64, learning_rate=0.001)

In [ ]:
# Presenting the summary of the model

model.summary()

In [ ]:
# Training the model for 50 epochs

epochs = 50

model_history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plotting the loss function from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['mae'], label = 'train mae')
plt.plot(model_history.history['val_mae'], label = 'test mae')
plt.title('History of the Loss Function of the Model')
plt.legend(['train mae', 'test mae'])
plt.show()

In [ ]:
# Plotting R^2 score from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['r_square'], label = 'train r^2')
plt.plot(model_history.history['val_r_square'], label = 'test r^2')
plt.title(r'$R^2$ of the Train and Test Subsets')
plt.legend([r'train $R^2$', r'test $R^2$'])
plt.show()

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Fully Connected NN - II. ReLu</p>

In [ ]:
df2=df.filter(['Store','Dept','Size','Week','Weekly_Sales'])

# Defining the features and the target of the model

features = df2.columns[:-1]
target = df2.columns[-1]

X = df2[features].values
y = df2[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
# Creating a simple function that builds a fully connected neural network

def create_model(n_neurons, learning_rate):
    model2 = Sequential()
    model2.add(Dense(n_neurons, input_dim=X_train.shape[1], activation='relu')) # Layer 1 + activation
    model2.add(Dense(n_neurons, activation='relu'))                             # Layer 2 + activation
    model2.add(Dense(n_neurons, activation='relu'))                             # Layer 3 + activation
    model2.add(Dense(len(y_train.shape)))                                          # Output layer
    adam = Adam(learning_rate=learning_rate)                                      # Choice of optimizer
    # Compiling the model by specifying the choice of loss function ('mae'), and performance metrics
    model2.compile(loss='mae', optimizer=adam, metrics=['mae', tfa.metrics.r_square.RSquare()])
    return model2

In [ ]:
# Creating a model with 64 neurons and learning rate 0.001

#del model2
model2 = create_model(n_neurons=64, learning_rate=0.001)

In [ ]:
# Presenting the summary of the model

model2.summary()

In [ ]:
# Training the model for 25 epochs

epochs = 25

model_history = model2.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plotting the loss function from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['mae'], label = 'train mae')
plt.plot(model_history.history['val_mae'], label = 'test mae')
plt.title('History of the Loss Function of the Model')
plt.legend(['train mae', 'test mae'])
plt.show()

In [ ]:
# Plotting R^2 score from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['r_square'], label = 'train r^2')
plt.plot(model_history.history['val_r_square'], label = 'test r^2')
plt.title(r'$R^2$ of the Train and Test Subsets')
plt.legend([r'train $R^2$', r'test $R^2$'])
plt.show()

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Fully Connected NN - III. TanH, 128 Neurons</p>

In [ ]:
df2=df.filter(['Store','Dept','Size','Week','Weekly_Sales'])

# Defining the features and the target of the model

features = df2.columns[:-1]
target = df2.columns[-1]

X = df2[features].values
y = df2[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
# Creating a simple function that builds a fully connected neural network

def create_model(n_neurons, learning_rate):
    model3 = Sequential()
    model3.add(Dense(n_neurons, input_dim=X_train.shape[1], activation='tanh')) # Layer 1 + activation
    model3.add(Dense(n_neurons, activation='tanh'))                             # Layer 2 + activation
    model3.add(Dense(n_neurons, activation='tanh'))                             # Layer 3 + activation
    model3.add(Dense(len(y_train.shape)))                                          # Output layer
    adam = Adam(learning_rate=learning_rate)                                      # Choice of optimizer
    # Compiling the model by specifying the choice of loss function ('mae'), and performance metrics
    model3.compile(loss='mae', optimizer=adam, metrics=['mae', tfa.metrics.r_square.RSquare()])
    return model3

In [ ]:
# Creating a model with 128 neurons and learning rate 0.001

#del model3
model3 = create_model(n_neurons=128, learning_rate=0.001)

In [ ]:
# Presenting the summary of the model

model3.summary()

In [ ]:
# Training the model for 25 epochs

epochs = 25

model_history = model3.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plotting the loss function from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['mae'], label = 'train mae')
plt.plot(model_history.history['val_mae'], label = 'test mae')
plt.title('History of the Loss Function of the Model')
plt.legend(['train mae', 'test mae'])
plt.show()

In [ ]:
# Plotting R^2 score from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['r_square'], label = 'train r^2')
plt.plot(model_history.history['val_r_square'], label = 'test r^2')
plt.title(r'$R^2$ of the Train and Test Subsets')
plt.legend([r'train $R^2$', r'test $R^2$'])
plt.show()

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Fully Connected NN - IV. TanH, 64 Neurons, Learning Rate 0.0001</p>

In [ ]:
df2=df.filter(['Store','Dept','Size','Week','Weekly_Sales'])

# Defining the features and the target of the model

features = df2.columns[:-1]
target = df2.columns[-1]

X = df2[features].values
y = df2[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
# Creating a simple function that builds a fully connected neural network

def create_model(n_neurons, learning_rate):
    model4 = Sequential()
    model4.add(Dense(n_neurons, input_dim=X_train.shape[1], activation='tanh')) # Layer 1 + activation
    model4.add(Dense(n_neurons, activation='tanh'))                             # Layer 2 + activation
    model4.add(Dense(n_neurons, activation='tanh'))                             # Layer 3 + activation
    model4.add(Dense(len(y_train.shape)))                                          # Output layer
    adam = Adam(learning_rate=learning_rate)                                      # Choice of optimizer
    # Compiling the model by specifying the choice of loss function ('mae'), and performance metrics
    model4.compile(loss='mae', optimizer=adam, metrics=['mae', tfa.metrics.r_square.RSquare()])
    return model4

In [ ]:
# Creating a model with 64 neurons and learning rate 0.0001

#del model4
model4 = create_model(n_neurons=64, learning_rate=0.0001)

In [ ]:
# Presenting the summary of the model

model4.summary()

In [ ]:
# Training the model for 25 epochs

epochs = 25

model_history = model4.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plotting the loss function from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['mae'], label = 'train mae')
plt.plot(model_history.history['val_mae'], label = 'test mae')
plt.title('History of the Loss Function of the Model')
plt.legend(['train mae', 'test mae'])
plt.show()

In [ ]:
# Plotting R^2 score from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['r_square'], label = 'train r^2')
plt.plot(model_history.history['val_r_square'], label = 'test r^2')
plt.title(r'$R^2$ of the Train and Test Subsets')
plt.legend([r'train $R^2$', r'test $R^2$'])
plt.show()

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Fully Connected NN - V. One Store, One Dept</p>

In [ ]:
av_sales_plotter('Store')

In [ ]:
av_sales_plotter('Dept')

In [ ]:
df2=df.filter(['Store','Dept','Size','Week','Weekly_Sales'])
df2_1=df2[df2['Store']==20] #limit to store
df2_2=df2_1[df2_1['Dept']==92] #limit to store and dept

df2_2.head(5)

In [ ]:
features = df2_2.columns[:-1]
target = df2_2.columns[-1]

X = df2_2[features].values
y = df2_2[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
# Creating a simple function that builds a fully connected neural network

def create_model(n_neurons, learning_rate):
    model5 = Sequential()
    model5.add(Dense(n_neurons, input_dim=X_train.shape[1], activation='tanh')) # Layer 1 + activation
    model5.add(Dense(n_neurons, activation='tanh'))                             # Layer 2 + activation
    model5.add(Dense(n_neurons, activation='tanh'))                             # Layer 3 + activation
    model5.add(Dense(len(y_train.shape)))                                          # Output layer
    adam = Adam(learning_rate=learning_rate)                                      # Choice of optimizer
    # Compiling the model by specifying the choice of loss function ('mae'), and performance metrics
    model5.compile(loss='mae', optimizer=adam, metrics=['mae', tfa.metrics.r_square.RSquare()])
    return model5

In [ ]:
# Creating a model with 64 neurons and learning rate 0.001

#del model5
model5 = create_model(n_neurons=64, learning_rate=0.001)

In [ ]:
# Presenting the summary of the model

model5.summary()

In [ ]:
# Training the model for 50 epochs

epochs = 50

model_history = model5.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plotting the loss function from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['mae'], label = 'train mae')
plt.plot(model_history.history['val_mae'], label = 'test mae')
plt.title('History of the Loss Function of the Model')
plt.legend(['train mae', 'test mae'])
plt.show()

In [ ]:
# Plotting R^2 score from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['r_square'], label = 'train r^2')
plt.plot(model_history.history['val_r_square'], label = 'test r^2')
plt.title(r'$R^2$ of the Train and Test Subsets')
plt.legend([r'train $R^2$', r'test $R^2$'])
plt.show()

# <p style="background-color:coral;font-family:newtimeroman;color:white;font-size:120%;text-align:center;border-radius:20px 60px;">Fully Connected NN - VI. One Store, One Dept, Relu 128 Neurons</p>

In [ ]:
features = df2_2.columns[:-1]
target = df2_2.columns[-1]

X = df2_2[features].values
y = df2_2[target].values

# Breaking the data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

In [ ]:
# Creating a simple function that builds a fully connected neural network

def create_model(n_neurons, learning_rate):
    model6 = Sequential()
    model6.add(Dense(n_neurons, input_dim=X_train.shape[1], activation='relu')) # Layer 1 + activation
    model6.add(Dense(n_neurons, activation='relu'))                             # Layer 2 + activation
    model6.add(Dense(n_neurons, activation='relu'))                             # Layer 3 + activation
    model6.add(Dense(len(y_train.shape)))                                          # Output layer
    adam = Adam(learning_rate=learning_rate)                                      # Choice of optimizer
    # Compiling the model by specifying the choice of loss function ('mae'), and performance metrics
    model6.compile(loss='mae', optimizer=adam, metrics=['mae', tfa.metrics.r_square.RSquare()])
    return model6

In [ ]:
# Creating a model with 64 neurons and learning rate 0.001

#del model6
model6 = create_model(n_neurons=64, learning_rate=0.001)

In [ ]:
# Presenting the summary of the model

model6.summary()

In [ ]:
# Training the model for 25 epochs

epochs = 50

model_history = model6.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plotting the loss function from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['mae'], label = 'train mae')
plt.plot(model_history.history['val_mae'], label = 'test mae')
plt.title('History of the Loss Function of the Model')
plt.legend(['train mae', 'test mae'])
plt.show()

In [ ]:
# Plotting R^2 score from model's history

plt.figure(figsize=(8,5))
plt.plot(model_history.history['r_square'], label = 'train r^2')
plt.plot(model_history.history['val_r_square'], label = 'test r^2')
plt.title(r'$R^2$ of the Train and Test Subsets')
plt.legend([r'train $R^2$', r'test $R^2$'])
plt.show()